In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from sympy.physics.quantum import TensorProduct
from matplotlib import cm

In [ ]:
#create the Pauli matrices
s0 = sp.eye(2)
sx = sp.Matrix([[0, 1], [1, 0]])
sy = sp.Matrix([[0, -sp.I], [sp.I, 0]])
sz = sp.Matrix([[1, 0], [0, -1]])

In [ ]:
#create parameters and the Pauli matrices 
kx_sym, ky_sym, kz_sym = sp.symbols('k_x k_y k_z', real = True)
ksymbols = [kx_sym, ky_sym, kz_sym]
alpha_sym = sp.symbols('alpha', real = True, positive = True)
gamma_z, lambda_z = sp.symbols('gamma_z lambda_z', real = True)

In [ ]:
H_dartboard = - sp.sin(kx_sym) * sp.sin(2*ky_sym) * sx
H_dartboard += sp.sin(2*kx_sym) * sp.sin(ky_sym) * sy 
H_dartboard += (alpha_sym + sp.cos(2*kx_sym) + sp.cos(2*ky_sym)) * sz

In [ ]:
H_layered = sp.Matrix(np.zeros((4,4)))
H_layered += TensorProduct(sz,H_dartboard)
H_layered += TensorProduct(sy,s0) * lambda_z * sp.sin(kz_sym)
H_layered += TensorProduct(sx,s0) * (gamma_z + lambda_z * sp.cos(kz_sym))

In [ ]:
H_layered

In [ ]:
#define system size
Nx = 30
Ny = 30
Nz = 30
Nbands = 4
Nocc = 2

params = {}
params["Nx"] = Nx
params["Ny"] = Ny
params["Nz"] = Nz
params["Nbands"] = Nbands
params["Nocc"] = Nocc

In [ ]:
Kxs = np.linspace(0, 2*np.pi, Nx, endpoint = False)
Kys = np.linspace(0, 2*np.pi, Ny, endpoint = False)
Kzs = np.linspace(0, 2*np.pi, Nz, endpoint = False)

In [ ]:
#fix the parameters of the model
H_fixparam = H_layered.subs({alpha_sym : 1, gamma_z: 0.5, lambda_z: 1})

In [ ]:
#calculate the eigenvalues and eigenstates of the system
hfunc = sp.lambdify((kx_sym,ky_sym,kz_sym), H_fixparam, modules = "numpy")

In [ ]:
eigenvalues = np.zeros((Nx,Ny,Nz,Nbands))
eigenstates = np.zeros((Nx,Ny,Nz,Nbands,Nbands), dtype = np.complex128)

for i in range(Nx):
    for j in range(Ny):
        for k in range(Nz):
            vals, vecs = np.linalg.eigh(hfunc(Kxs[i],Kys[j],Kzs[k]))

            ind = np.argsort(vals)

            eigenvalues[i,j,k] = vals[ind]
            eigenstates[i,j,k] = vecs[:,ind]

In [ ]:
Links_z = np.zeros((Nx,Ny,Nz,Nocc,Nocc), dtype = np.complex128)

for i in range(Nx):
    for j in range(Ny):
        for k in range(Nz): 
            ol = eigenstates[i,j,(k+1)%Nz,:,:Nocc].conj().T @ eigenstates[i,j,k,:,:Nocc]
            S, _, V = np.linalg.svd(ol)

            Links_z[i,j,k] = S @ V

Wilsonloops = np.zeros((Nx,Ny,Nocc,Nocc), dtype = np.complex128)

for i in range(Nx):
    for j in range(Ny):
        W = np.eye(Nocc).astype(np.complex128)
        for k in range(Nz):
            W = Links_z[i,j,k] @ W

        Wilsonloops[i,j] = W

nuvals = np.zeros((Nx,Ny,Nocc))
nuvecs = np.zeros((Nx,Ny,Nocc,Nocc), dtype = np.complex128)

for i in range(Nx):
    for j in range(Ny):
        vals, vecs = np.linalg.eig(Wilsonloops[i,j])
        angles = np.angle(vals) / (2 * np.pi)

        ind = np.argsort(angles)

        nuvals[i,j] = angles[ind]
        nuvecs[i,j] = vecs[:,ind]

In [ ]:
fig, ax = plt.subplots(1,1, subplot_kw={"projection": "3d"}, figsize = (4,3))
plt.subplots_adjust(bottom=0.1, right=1, top=2)

X, Y = np.meshgrid(Kxs, Kys)

ax.plot_surface(X.T, Y.T, nuvals[:,:,0], cmap=cm.viridis, linewidth=0, antialiased=False, vmin = -0.15, vmax = 0.15, alpha = 1.0)
ax.plot_surface(X.T, Y.T, nuvals[:,:,1], cmap=cm.viridis, linewidth=0, antialiased=False, vmin = -0.15, vmax = 0.15, alpha = 1.0)

ax.set_xlabel(r"$k_x$", fontsize = 16, labelpad = 8)
ax.set_ylabel(r"$k_y$", fontsize = 16, labelpad = 8)
ax.set_xticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi], [r"$0$", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"], fontsize = 14)
ax.set_yticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi], [r"$0$", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"], fontsize = 14)
ax.set_zticks([-0.1, 0, 0.1], [r"$-0.1$", r"$0$", r"$0.1$"], fontsize = 14)
ax.set_title(r"$\nu_z$", x = 1.02, y = 0.85, fontsize = 16)
ax.view_init(15, 120)
plt.show()